In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import anndata
import scanpy.external as sce
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
COPD = sc.read("processed_data_py/COPD_newdata_0808.h5ad")
mes = COPD[COPD.obs["cellclass"] == "Mesenchymal"].copy()

In [ ]:
sc.pp.highly_variable_genes(mes, n_top_genes=2000, flavor = "seurat")
mes.raw = mes
mes = mes[:, mes.var.highly_variable]
sc.pp.regress_out(mes, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(mes)
sc.tl.pca(mes, n_comps = 50)
sc.external.pp.bbknn(mes, batch_key = "sample")
sc.tl.umap(mes)
sc.pl.umap(mes, color=["Group"], size=1)
sc.pp.neighbors(mes, n_neighbors=30, n_pcs=50)
sc.tl.leiden(mes,resolution = 1.5)
sc.pl.umap(mes, color=["leiden"],size=1, legend_loc='on data')

In [ ]:
sc.pl.dotplot(mes,{"mesthelial":["EPCAM","SCGB3A2","SCGB1A1","FOXJ1","SFTPC",],
                        "Mesenchymal":["COL1A2","MFAP4","DCN","PDGFRA","SERPINF1",
                                       "SFRP2","PI16","RSPO3","GPC3","ITGA8","SPINT2","FGFR4",
                              "ASPN","WIF1","SCX","LGR6","CTHRC1","POSTN","COL11A1","CRABP1","TNNT2","RERGL","CCL21","RGS5",
                                       "ACTA2","MYH11",
                              "COX4I2","LAMC3","ACTG2","DES","UPK3B","ITLN1"],
                        "Endothleial":["PECAM1","CDH5"],
                        "Immune":["PTPRC"],
                       "MKI67":["TOP2A"]}, 
                  groupby="leiden",dendrogram=False,standard_scale="var",cmap='Spectral_r')

In [ ]:
mes.obs["celltype"] = ""
mes.obs.loc[mes.obs["leiden"].isin(["11","13","15","17","22","23","24","26","27","29","32"]),"celltype"] = "Doublets"
mes.obs.loc[mes.obs["leiden"].isin(["0","3","6","7","8","14","16","18","19","25","28"]),"celltype"] = "Adventitial fibroblast"
mes.obs.loc[mes.obs["leiden"].isin(["1","2","4","9","28","30","31","33"]),"celltype"] = "Alveolar fibroblast"
mes.obs.loc[mes.obs["leiden"].isin(["12"]),"celltype"] = "Myofibroblast"
mes.obs.loc[mes.obs["leiden"].isin(["21"]),"celltype"] = "Fibromyocyte"
mes.obs.loc[mes.obs["leiden"].isin(["5"]),"celltype"] = "Pericyte"
mes.obs.loc[mes.obs["leiden"].isin(["20"]),"celltype"] = "VSM"
mes.obs.loc[mes.obs["leiden"].isin(["10"]),"celltype"] = "ASM"
sc.pl.umap(mes, color=["celltype"],size=1, legend_loc='on data', legend_fontsize= 8)

In [ ]:
mes_meta = mes.obs[["celltype"]]
mes_meta.to_csv("tables/mes_meta.csv")
mes = mes.raw.to_adata()

In [ ]:
mes = mes[mes.obs["celltype"] != "Doublets"].copy()
sc.tl.pca(mes, n_comps = 50)
sc.external.pp.bbknn(mes, batch_key = "sample")
sc.tl.umap(mes)
sc.pl.umap(mes, color=["celltype"],size=1, legend_loc='on data')

In [ ]:
sc.pl.dotplot(mes,{"Adventitial fibroblast" : ["SERPINF1","SFRP2","PI16"], "Alveolar fibroblast" : ["GPC3","ITGA8","SPINT2"],
                   "Myofibroblast" : ["ASPN","ITGBL1","PDGFRA"], "Fibromyocyte" : ["ZNF385D","HPSE2","LGR6"],
                   "ASM" : ["TAGLN","ACTA2","TNNT2"], "VSM" : ["RGS5","SLIT3"], "Pericyte" :["LAMC3","COX4I2"] },
                  groupby="celltype",dendrogram=False,standard_scale="var",cmap = "RdBu_r")

In [ ]:
mes.obs['UMAP1'] = mes.obsm['X_umap'][:, 0]
mes.obs['UMAP2'] = mes.obsm['X_umap'][:, 1]
mes.obs.to_csv("./tables/mes_meta_umap.csv")
mes.write("./processed_data_py/COPD_mes.h5ad")